In [1]:
%run "DataPreparation.ipynb";
%run "Modeling.ipynb";

In [2]:
# Setup
images_dir= "C:/Users/sowmi/Python scripts/hurricane/images" #images
json_dir = "C:/Users/sowmi/Python scripts/hurricane/labels" #json

# Loading JSON data into dataframe for building localization
hurricane_pre_df, hurricane_post_df, pre_hurricane_images, post_hurricane_images = prepare_data(images_dir, json_dir);

Retrieving pre and post disaster images from: C:/Users/sowmi/Python scripts/hurricane/images

Total pre-disaster images: 1219

Total post-disaster images: 1219


In [3]:
pre_resized_img, pre_mask_resized_img, polygon_data= preprocess_data(hurricane_pre_df, hurricane_post_df, images_dir);

In [4]:
X_train, X_valid, y_train, y_valid=split_data(pre_resized_img, pre_mask_resized_img);
#For Classification model
data_df = pd.DataFrame(polygon_data)
data_df.head()
final_df = data_df.drop(['polygon_points'], axis=1)
# Renaming columns
final_df = final_df.rename(columns={'uid': 'building_uid',
                                    'damage_type': 'labels'})
# Convert labels to strings
final_df['labels'] = final_df['labels'].astype(str)
final_df.head()


(1219, 256, 256, 3)
(1219, 256, 256, 1)


,building_uid,labels
0,82799c02-ee88-452a-a615-7df5e64560e8.png,0
1,bbc71c55-2b37-4582-842e-27a70431dd56.png,0
2,243c58e3-d7f9-4feb-a879-11ef7fc64a75.png,0
3,aa1a6143-efc6-4d66-b5d6-2e827573f694.png,0
4,2b9f0812-faa6-4635-a6a2-1b214c273a7f.png,0


### Class Imbalance 

In [5]:
final_df['labels'].value_counts()

0    34846
1    14550
2    13330
3     3359
Name: labels, dtype: int64

In [6]:
from sklearn.utils import resample

# Assuming 'df' has columns 'images' (file paths or data) and 'labels' (target class)

# Separate the majority and minority classes
min_class_size = final_df['labels'].value_counts().min()
balanced_df = []

# Iterate over each class and undersample the majority classes
for label in final_df['labels'].unique():
    class_subset = final_df[final_df['labels'] == label]
    undersampled_subset = resample(class_subset, 
                                   replace=False,     # No replacement
                                   n_samples=min_class_size,  # Match minority class size
                                   random_state=42)  # Reproducibility
    balanced_df.append(undersampled_subset)

# Combine all classes back into a single DataFrame
balanced_df = pd.concat(balanced_df)

balanced_df = balanced_df.sample(frac=1, random_state=42).reset_index(drop=True)
balanced_df['labels'].value_counts()

0    3359
1    3359
2    3359
3    3359
Name: labels, dtype: int64

In [7]:
balanced_df.shape

(13436, 2)

In [8]:
train_df, valid_df = classification_split_data(balanced_df)

(10748, 2)
(2688, 2)


In [9]:
#ONLY IF TRAINING IS NEEDED!
# train_FCN_model();
# train_unet_model();
train_generator, val_generator = data_gen(target_size=(224,224), batch_size=32) # Mobilenet is trained on 224,224 images
#train_ResNet50_model(train_generator, val_generator, batch_size=32);
# train_MobileNet_model(train_generator, val_generator);
# train_CNN_model(train_generator, val_generator);

Found 10748 validated image filenames belonging to 4 classes.
Found 2688 validated image filenames belonging to 4 classes.


# FCN Model

In [7]:
fcn_model = tf.keras.models.load_model('model/FCN/FCN_model.keras')
fcn_model.summary()
# Making predictions
fcn_model_mask = fcn_model.predict(X_valid)
# Model Evaluation
loss, accuracy = fcn_model.evaluate(X_valid, y_valid)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 256, 256, 3)]     0         
                                                                 
 conv2d_31 (Conv2D)          (None, 256, 256, 64)      1792      
                                                                 
 conv2d_32 (Conv2D)          (None, 256, 256, 64)      36928     
                                                                 
 conv2d_33 (Conv2D)          (None, 128, 128, 128)     73856     
                                                                 
 conv2d_34 (Conv2D)          (None, 128, 128, 128)     147584    
                                                                 
 conv2d_transpose_6 (Conv2DT  (None, 256, 256, 64)     73792     
 ranspose)                                                       
                                                           

# Unet Model

In [5]:
unet_model = tf.keras.models.load_model('model/Unet/Unet_model.keras')
unet_model.summary()
# Making predictions
unet_model_mask = unet_model.predict(X_valid)
# Model Evaluation
loss, accuracy = unet_model.evaluate(X_valid, y_valid)
print(f"Validation Loss: {loss:.4f}")
print(f"Validation Accuracy: {accuracy:.4f}")

Model: "U-Net"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 256, 256, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d_6 (Conv2D)              (None, 256, 256, 64  1792        ['input_5[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization_6 (BatchNo  (None, 256, 256, 64  256        ['conv2d_6[0][0]']               
 rmalization)                   )                                                             

# MobileNet Model

In [6]:
test_datagen = ImageDataGenerator(rescale=1/255.)

test_generator = test_datagen.flow_from_dataframe(
    dataframe=test_df,
    directory=cropped_dir,
    x_col='building_uid',
    y_col='labels',
    target_size=(224, 224),
    batch_size=32,
    shuffle=False,
    seed=123,
    class_mode="categorical"
)


Found 1322 validated image filenames belonging to 4 classes.
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 128, 128, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 128, 128, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 128, 128, 32  128        ['conv2d[0][0]']                 
 alization)                     )

In [20]:
with tf.device('/cpu:0'):
    mobilenet_model = tf.keras.models.load_model('model/MobileNet/MobileNet_model.keras')
    mobilenet_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [21]:
# Making predictions
# print(test_generator)
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils

img_path= '../cropped_images/'
filename = 'ffe7fa73-ecf8-48f2-9068-f690eee0a533.png'
img = image.load_img(img_path+filename, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array_expanded_dims = np.expand_dims(img_array, axis=0)

class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

processed_image= tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)
# img_array_expanded_dims = np.expand_dims(test_generator[0], axis=0)
predictions = mobilenet_model.predict(processed_image)
output_class=class_names[np.argmax(predictions)]

print(output_class)
# results = imagenet_utils.decode_predictions(predictions)
# print(results)
# Model Evaluation
# loss, accuracy = unet_model.evaluate(X_valid, y_valid)
# print(f"Validation Loss: {loss:.4f}")
# print(f"Validation Accuracy: {accuracy:.4f}")

1/1 [==============================] - 1s 750ms/step
0


In [10]:
with tf.device('/cpu:0'):
    resnet50_model = tf.keras.models.load_model('model/Resnet50/Resnet50_model.keras')
    resnet50_model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv2d (Conv2D)                (None, 224, 224, 32  896         ['input_1[0][0]']                
                                )                                                                 
                                                                                                  
 batch_normalization (BatchNorm  (None, 224, 224, 32  128        ['conv2d[0][0]']                 
 alization)                     )                                                             

In [11]:
from keras.applications.mobilenet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications import imagenet_utils

img_path= '../cropped_images/'
filename = 'fff86503-2967-4603-acd6-adf6344b65d8.png'
img = image.load_img(img_path+filename, target_size=(224, 224))
img_array = image.img_to_array(img)
img_array_expanded_dims = np.expand_dims(img_array, axis=0)

class_indices = train_generator.class_indices
class_names = list(class_indices.keys())

processed_image= tf.keras.applications.mobilenet.preprocess_input(img_array_expanded_dims)
# img_array_expanded_dims = np.expand_dims(test_generator[0], axis=0)
predictions = resnet50_model.predict(processed_image)
output_class=class_names[np.argmax(predictions)]

print(output_class)

1/1 [==============================] - 6s 6s/step
2
